In [4]:
import traceback
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from pytket.extensions.qiskit import qiskit_to_tk
from pytket.qasm import circuit_to_qasm_str
from qiskit.qasm2 import dumps

# Section: Circuit
qr = QuantumRegister(3, name='qr')
cr = ClassicalRegister(3, name='cr')
qc = QuantumCircuit(qr, cr, name='qc')

# Apply gate operations
qc.cry(1.5, qr[1], qr[2])

# Export to qasm

# single decomposition
circuit_to_export = qc.decompose()
print(circuit_to_export.draw(output='text'))
#                   ┌─┐
# qr_0: ────────────┤M├─────────────────────────────
#                   └╥┘                       ┌─┐
# qr_1: ─────────────╫───■─────────────────■──┤M├───
#       ┌──────────┐ ║ ┌─┴─┐┌───────────┐┌─┴─┐└╥┘┌─┐
# qr_2: ┤ Ry(0.75) ├─╫─┤ X ├┤ Ry(-0.75) ├┤ X ├─╫─┤M├
#       └──────────┘ ║ └───┘└───────────┘└───┘ ║ └╥┘
# cr: 3/═════════════╩═════════════════════════╩══╩═
#                    0                         1  2

# two decompositions
circuit_to_export_2_decompose = qc.decompose().decompose()
print(circuit_to_export_2_decompose.draw(output='text'))
#                      ┌─┐
# qr_0: ───────────────┤M├────────────────────────────────
#                      └╥┘                          ┌─┐
# qr_1: ────────────────╫───■────────────────────■──┤M├───
#       ┌─────────────┐ ║ ┌─┴─┐┌──────────────┐┌─┴─┐└╥┘┌─┐
# qr_2: ┤ R(0.75,π/2) ├─╫─┤ X ├┤ R(-0.75,π/2) ├┤ X ├─╫─┤M├
#       └─────────────┘ ║ └───┘└──────────────┘└───┘ ║ └╥┘
# cr: 3/════════════════╩════════════════════════════╩══╩═
#                       0                            1  2

print("Via Qiskit - Two Decomposition")
qasm_str_qiskit = dumps(circuit_to_export_2_decompose)
print(qasm_str_qiskit)
# OPENQASM 2.0;
# include "qelib1.inc";
# gate r(param0,param1) q0 { u3(param0,param1 - pi/2,pi/2 - param1) q0; }
# qreg qr[3];
# creg cr[3];
# r(2.5277165,pi/2) qr[2];
# cx qr[1],qr[2];
# r(-2.5277165,pi/2) qr[2];
# cx qr[1],qr[2];
# measure qr[0] -> cr[0];
# measure qr[1] -> cr[1];
# measure qr[2] -> cr[2];

print("Via TKET - Single Decomposition")
tket_circ = qiskit_to_tk(circuit_to_export)
qasm_str_tket = circuit_to_qasm_str(tket_circ, header='qelib1')
print(qasm_str_tket)
# OPENQASM 2.0;
# include "qelib1.inc";
# qreg qr[3];
# creg cr[3];
# measure qr[0] -> cr[0];
# ry(0.238732414637843*pi) qr[2];
# cx qr[1],qr[2];
# ry(3.761267585362157*pi) qr[2];
# cx qr[1],qr[2];
# measure qr[1] -> cr[1];
# measure qr[2] -> cr[2];


print("Via TKET - Two Decompositions")
# tket_circ = qiskit_to_tk(circuit_to_export_2_decompose)
# qasm_str_tket = circuit_to_qasm_str(tket_circ, header='qelib1')
# print(qasm_str_tket)
# # QASMUnsupportedError: Cannot print command of type: PhasedX(0.238732, 0.5)


# print stack trace
try:
    tket_circ = qiskit_to_tk(circuit_to_export_2_decompose)
    qasm_str_tket = circuit_to_qasm_str(tket_circ, header='qelib1')
    print(qasm_str_tket)
except Exception as e:
    print(traceback.format_exc())
    # Traceback (most recent call last):
    #   File "qiskit_to_tket.py", line 72, in <module>
    #     qasm_str_tket = circuit_to_qasm_str(tket_circ, header='qelib1')
    #   File "/Users/ryan/venv/pytket/lib/python3.8/site-packages/pytket/qasm/_qasm.py", line 51, in circuit_to_qasm_str
    #     raise QASMUnsupportedError(f"Cannot print command of type: {cmd.op}")
    # pytket.qasm._qasm.QASMUnsupportedError: Cannot print command of type: PhasedX(0.238732, 0.5)

                                         
qr_0: ───────────────────────────────────
                                         
qr_1: ──────────────■─────────────────■──
      ┌──────────┐┌─┴─┐┌───────────┐┌─┴─┐
qr_2: ┤ Ry(0.75) ├┤ X ├┤ Ry(-0.75) ├┤ X ├
      └──────────┘└───┘└───────────┘└───┘
cr: 3/═══════════════════════════════════
                                         
                                               
qr_0: ─────────────────────────────────────────
                                               
qr_1: ─────────────────■────────────────────■──
      ┌─────────────┐┌─┴─┐┌──────────────┐┌─┴─┐
qr_2: ┤ R(0.75,π/2) ├┤ X ├┤ R(-0.75,π/2) ├┤ X ├
      └─────────────┘└───┘└──────────────┘└───┘
cr: 3/═════════════════════════════════════════
                                               
Via Qiskit - Two Decomposition
OPENQASM 2.0;
include "qelib1.inc";
gate r(param0,param1) q0 { u3(param0,param1 - pi/2,pi/2 - param1) q0; }
qreg qr[3];
creg cr[3];
r(0.75,pi/2) qr[2];
cx qr[1

In [2]:
import qiskit
import pytket
print(f"Qiskit version: {qiskit.__version__}")
print(f"Pytket version: {pytket.__version__}")

Qiskit version: 1.2.4
Pytket version: 1.33.1
